In [ ]:
import torch
import adapters

from transformers import AutoModel, AutoModelForCausalLM, AutoConfig
from adapters import LlamaAdapterModel
from transformers import AutoTokenizer
from adapters.models.llama.adapter_model import LlamaAdapterModel

In [ ]:
device = torch.device("cuda")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")

#
config = AutoConfig.from_pretrained("meta-llama/Llama-2-7b-hf")
#config.load_in_8bit = True

#
model = LlamaAdapterModel.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True)
model.add_causal_lm_head("lm_head")

#
model_ref = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True)


In [ ]:
model

In [ ]:
model.heads.lm_head[0].weight = model_ref.lm_head.weight
del model_ref

In [ ]:
model_ref.lm_head.weight

In [ ]:
model.eval()
for name, par in model.named_parameters():
    par.requires_grad = False

#model.add_adapter("adapter", config="lora", set_active=True)
#model.to(device)

In [ ]:
model

In [ ]:
model.model.layers[0].attention_adapters

In [ ]:
adapters.init(model)

In [ ]:
model.to(device)

In [ ]:
toked = tokenizer.encode_plus(
    "Sentence: I feel good. Sentiment of the previous sentence: Positive. Sentence: I feel bad. Sentiment of the previous sentence: ", return_tensors="pt")
input_ids = toked["input_ids"]
attn_mask = toked["attention_mask"]
input_ids = input_ids.to(device)
attn_mask = attn_mask.to(device)
print(input_ids.shape)
print(input_ids.device)

In [ ]:
out_ = model.generate(
    inputs=input_ids,
    max_length=32,
    min_length=2,
    num_beams=5
)

In [ ]:
out_

In [ ]:
tokenizer.batch_decode(out_)

In [ ]:
model.add_adapter("adapter", "pfeiffer", set_active=True)
model.train()
model.train_adapter("adapter")

In [ ]:
model.model.layers[0].output_adapters.adapters.adapter.adapter_down[0].weight.device

In [ ]:
model.to(device)

In [ ]:
for name, module in model.named_modules():
    if "adapter" in name:
        module.to(device)

In [ ]:
model.model.layers[0].output_adapters.adapters.adapter.adapter_down[0].weight.device

In [ ]:
model(input_ids=input_ids)